In [7]:
import numpy as np

def finite_diff_grad(estimator, circuit, observable, x, eps=1e-6, precision=None):
    x = np.asarray(x, dtype=float)
    pubs = []
    for k in range(len(x)):
        xp = x.copy(); xp[k] += eps
        xm = x.copy(); xm[k] -= eps
        pubs.append((circuit, observable, [xp.tolist()], precision))
        pubs.append((circuit, observable, [xm.tolist()], precision))

    result = estimator.run(pubs).result()
    evs = [float(result[i].data.evs[0]) for i in range(len(pubs))]
    grad = np.array([(evs[2*k] - evs[2*k + 1]) / (2 * eps) for k in range(len(x))], dtype=float)
    return grad


In [8]:
import numpy as np

def finite_diff_grad(estimator, circuit, observable, x, eps=1e-4):
   
    x = np.asarray(x, dtype=float)
    params = list(circuit.parameters)
    n = len(params)

    if x.shape != (n,):
        raise ValueError(f"x must have shape ({n},), got {x.shape}.")

    # 2n parameter sets in a single batch
    X = np.repeat(x[None, :], 2 * n, axis=0)
    for k in range(n):
        X[2 * k, k]     += eps
        X[2 * k + 1, k] -= eps

    pub = (circuit, [observable], X.tolist())
    res = estimator.run([pub]).result()

    evs = np.asarray(res[0].data.evs, dtype=float)
    if evs.ndim == 2:
        evs = evs[0]

    grad = (evs[0::2] - evs[1::2]) / (2.0 * eps)
    return grad


In [9]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import XXPlusYYGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator


# --- circuit with CRY + XXPlusYYGate ---
thetas = ParameterVector("θ", 2)
qc = QuantumCircuit(2)
qc.x(0)
qc.cry(thetas[0], 0, 1)
qc.append(XXPlusYYGate(thetas[1], beta=0.0), [0, 1])

# observable (example)
obs = SparsePauliOp.from_list([("ZZ", 1.0)])


estimator = StatevectorEstimator()


param_order = list(qc.parameters)  # important: matches parameter_values ordering
x = [0.1, 0.2]                     # values in that order
grad = finite_diff_grad(estimator, qc, obs, x)


print("parameter order:", param_order)
print("gradient:", grad)


parameter order: [ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1])]
gradient: [9.98334165e-02 1.11022302e-12]
